Un entrenador de fútbol 5 tine que elegir a su formación titular para jugar la final del torneo Las Cañas. Siempre juega con 1 arquera, 2 defensoras y 2 delanteras.
La lista de jugadoras disponibles, junto con su calidad en cada posición, se encuentran en la sigiuente tabla

In [1]:
import pandas as pd
from ortools.linear_solver import pywraplp

## Dataset

In [2]:
dataset = pd.DataFrame({
    'Arquera': [3,5,6,4,6,6,8,4],
    'Defensora': [8,5,5,7,7,5,10,9],
    'Delantera': [8,6,4,6,6,3,5,9]
})
dataset.index = ['Ana', 'Belén', 'Carla', 'Daniela', 'Erica', 'Francisca', 'Gabriela', 'Hilda']
dataset

,Arquera,Defensora,Delantera
Ana,3,8,8
Belén,5,5,6
Carla,6,5,4
Daniela,4,7,6
Erica,6,7,6
Francisca,6,5,3
Gabriela,8,10,5
Hilda,4,9,9


In [3]:
solver = pywraplp.Solver.CreateSolver('CBC')

## Variables

In [4]:
dataset.columns[1:].tolist()

['Defensora', 'Delantera']

In [5]:
posiciones = [('Arquera', 1), ('Defensora', 2), ('Delantera', 2)]
jugadoras = dataset.index.tolist()

In [6]:
variables = {}

for pos in posiciones:
    for jug in jugadoras:
        variable = solver.BoolVar(f'X_{pos[0]}_{jug}')
        variables[(pos[0], jug)] = variable

In [7]:
variables

{('Arquera', 'Ana'): X_Arquera_Ana,
 ('Arquera', 'Belén'): X_Arquera_Belén,
 ('Arquera', 'Carla'): X_Arquera_Carla,
 ('Arquera', 'Daniela'): X_Arquera_Daniela,
 ('Arquera', 'Erica'): X_Arquera_Erica,
 ('Arquera', 'Francisca'): X_Arquera_Francisca,
 ('Arquera', 'Gabriela'): X_Arquera_Gabriela,
 ('Arquera', 'Hilda'): X_Arquera_Hilda,
 ('Defensora', 'Ana'): X_Defensora_Ana,
 ('Defensora', 'Belén'): X_Defensora_Belén,
 ('Defensora', 'Carla'): X_Defensora_Carla,
 ('Defensora', 'Daniela'): X_Defensora_Daniela,
 ('Defensora', 'Erica'): X_Defensora_Erica,
 ('Defensora', 'Francisca'): X_Defensora_Francisca,
 ('Defensora', 'Gabriela'): X_Defensora_Gabriela,
 ('Defensora', 'Hilda'): X_Defensora_Hilda,
 ('Delantera', 'Ana'): X_Delantera_Ana,
 ('Delantera', 'Belén'): X_Delantera_Belén,
 ('Delantera', 'Carla'): X_Delantera_Carla,
 ('Delantera', 'Daniela'): X_Delantera_Daniela,
 ('Delantera', 'Erica'): X_Delantera_Erica,
 ('Delantera', 'Francisca'): X_Delantera_Francisca,
 ('Delantera', 'Gabriela'): 

## Constraints

In [8]:
for pos in posiciones:
    ct = solver.Constraint(pos[1], pos[1], f'Maxima cantidad de jugadores para la posición {pos[0]}')
    for jug in jugadoras:
        ct.SetCoefficient(variables[(pos[0], jug)], 1)

In [9]:
for jug in jugadoras:
    ct = solver.Constraint(0, 1, f'Maxima cantidad de posición para la jugadora: {jug}')
    for pos in posiciones:
        ct.SetCoefficient(variables[(pos[0], jug)], 1)

## Objective Function

In [10]:
objective = solver.Objective()
for jug in jugadoras:
    for pos in posiciones:
        calidad = dataset.loc[jug, pos[0]]
        objective.SetCoefficient(variables[(pos[0], jug)], int(calidad) )
objective.SetMaximization()

In [11]:
print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

 Generated by MPModelProtoExporter
   Name             : 
   Format           : Free
   Constraints      : 11
   Variables        : 24
     Binary         : 24
     Integer        : 0
     Continuous     : 0
Maximize
 Obj: +3 X_Arquera_Ana +5 X_Arquera_Belén +6 X_Arquera_Carla +4 X_Arquera_Daniela +6 X_Arquera_Erica +6 X_Arquera_Francisca +8 X_Arquera_Gabriela +4 X_Arquera_Hilda +8 X_Defensora_Ana +5 X_Defensora_Belén +5 X_Defensora_Carla +7 X_Defensora_Daniela +7 X_Defensora_Erica +5 X_Defensora_Francisca +10 X_Defensora_Gabriela +9 X_Defensora_Hilda +8 X_Delantera_Ana +6 X_Delantera_Belén +4 X_Delantera_Carla +6 X_Delantera_Daniela +6 X_Delantera_Erica +3 X_Delantera_Francisca +5 X_Delantera_Gabriela +9 X_Delantera_Hilda 
Subject to
 Maxima_cantidad_de_jugadores_para_la_posición_Arquera: +1 X_Arquera_Ana +1 X_Arquera_Belén +1 X_Arquera_Carla +1 X_Arquera_Daniela +1 X_Arquera_Erica +1 X_Arquera_Francisca +1 X_Arquera_Gabriela +1 X_Arquera_Hilda  = 1
 Maxima_cantidad_de_jugadores_para_

## Maximization

In [12]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')

eficiencia_total = solver.Objective().Value()
print(f'La eficiencia total es {eficiencia_total}')

El problema tiene una solución óptima
La eficiencia total es 40.0


In [17]:
variables[('Arquera', 'Ana')]

0.0

In [21]:
solution = dataset.copy()
for jug in jugadoras:
    for pos in posiciones:
        solution.loc[jug, pos[0]] = variables[pos[0], jug].solution_value()
solution

,Arquera,Defensora,Delantera
Ana,0,0,1
Belén,0,0,0
Carla,0,0,0
Daniela,0,1,0
Erica,0,0,0
Francisca,1,0,0
Gabriela,0,1,0
Hilda,0,0,1
